In [ ]:
# Install necessary libraries (Colab)
!pip install requests beautifulsoup4


In [ ]:
!pip install requests

In [ ]:
!pip install requests pandas

In [ ]:
!pip install requests  ipywidgets

In [ ]:
!pip install requests pillow

In [ ]:
from PIL import Image
from io import BytesIO

In [ ]:
import plotly.express as px


In [ ]:
from IPython.display import display, HTML

In [ ]:
import requests

In [ ]:
import pandas as pd

In [ ]:
import ipywidgets as widgets

In [ ]:
# Simple mapping
weather_styles = {
    "Sunny": {"icon": "https://img.icons8.com/color/96/000000/sun--v1.png", "bg": "#FFD700"},
    "Clear": {"icon": "https://img.icons8.com/color/96/000000/sun--v1.png", "bg": "#87CEFA"},
    "Partly cloudy": {"icon": "https://img.icons8.com/color/96/000000/partly-cloudy-day.png", "bg": "#B0C4DE"},
    "Cloudy": {"icon": "https://img.icons8.com/color/96/000000/cloud.png", "bg": "#A9A9A9"},
    "Overcast": {"icon": "https://img.icons8.com/color/96/000000/cloud.png", "bg": "#778899"},
    "Rain": {"icon": "https://img.icons8.com/color/96/000000/rain.png", "bg": "#4682B4"},
    "Snow": {"icon": "https://img.icons8.com/color/96/000000/snow.png", "bg": "#ADD8E6"},
    "Mist": {"icon": "https://img.icons8.com/color/96/000000/fog-day.png", "bg": "#C0C0C0"}
}

def get_style(condition):
    for key in weather_styles:
        if key.lower() in condition.lower():
            return weather_styles[key]
    return {"icon": "https://img.icons8.com/color/96/000000/cloud.png", "bg": "#FFFFFF"}  # default


In [ ]:
def get_weather(city):
    url = f"https://wttr.in/{city}?format=j1"
    try:
        r = requests.get(url)
        data = r.json()

        current = data['current_condition'][0]
        temp = current['temp_C']
        humidity = current['humidity']
        desc = current['weatherDesc'][0]['value']

        # Forecast (next 3 days)
        forecast = []
        for day in data['weather']:
            forecast.append([day['date'], int(day['avgtempC'])])
        forecast_df = pd.DataFrame(forecast, columns=['Date', 'AvgTempC'])

        return temp, humidity, desc, forecast_df
    except Exception as e:
        return None, None, f"Error: {e}", pd.DataFrame()


In [ ]:
# Widgets
city_input = widgets.Text(value="London", description="City:", placeholder="Enter city name")
button = widgets.Button(description="Get Weather", button_style="info")
output = widgets.Output()

def on_button_click(b):
    with output:
        output.clear_output()
        city = city_input.value.strip()
        if not city:
            print("❌ Please enter a city name.")
            return

        temp, humidity, desc, forecast_df = get_weather(city)
        if temp:
            style = get_style(desc)

            # Display weather with icon and colored background
            display(HTML(f"""
                <div style='background-color:{style['bg']}; padding:20px; border-radius:15px; text-align:center;'>
                    <h2>🌍 Weather in {city.title()}</h2>
                    <img src="{style['icon']}" width="80" height="80"><br>
                    <p>🌡️ <b>Temperature:</b> {temp} °C</p>
                    <p>💧 <b>Humidity:</b> {humidity}%</p>
                    <p>☁️ <b>Condition:</b> {desc}</p>
                </div>
            """))

            # Plot 3-day forecast
            fig = px.line(forecast_df, x='Date', y='AvgTempC',
                          title=f"3-Day Forecast for {city.title()}",
                          markers=True, line_shape='spline')
            fig.update_layout(template="plotly_dark", title_x=0.5)
            fig.show()
        else:
            print(desc)

button.on_click(on_button_click)

# Display UI
display(city_input, button, output)
